# Assignment 2

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("wenxingdi/data-expo-2009-airline-on-time-data")

print("Path to dataset files:", path)

KaggleApiHTTPError: 403 Client Error.

You don't have permission to access resource at URL: https://www.kaggle.com/datasets/d:/Muat Turun. The server reported the following issues: Permission 'datasets.get' was denied
Please make sure you are authenticated if you are trying to access a private resource or a resource requiring consent.

In [4]:
import os
os.getcwd()

'd:\\01. Education\\02. Master\\Semester 2\\Data-Management\\Project 2'

In [ ]:
import mlcroissant as mlc
import pandas as pd

# Fetch the Croissant JSON-LD
croissant_dataset = mlc.Dataset('https://www.kaggle.com/datasets/wenxingdi/data-expo-2009-airline-on-time-data/croissant/download')

# Check what record sets are in the dataset
record_sets = croissant_dataset.metadata.record_sets
print(record_sets)

# Fetch the records and put them in a DataFrame
record_set_df = pd.DataFrame(croissant_dataset.records(record_set=record_sets[0].uuid))
record_set_df.head()
